In [ ]:
import pandas as pd
import numpy as np

In [ ]:
OECD_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark',
                  'Finland', 'France', 'Germany', 'Greece', 'Iceland',
                  'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand',
                  'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland',
                  'United Kingdom', 'United States']

In [ ]:
# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [ ]:
# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(OECD_countries)][['year', 'country', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
#data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()

In [ ]:
data

,year,country,rgdpna,rkna,pop,emp,avh,labsh,rtfpna
560,1950,Australia,1.274760e+05,NaN,8.354106,3.429873,2170.923406,0.680492,NaN
561,1951,Australia,1.307183e+05,NaN,8.599923,3.523916,2150.846928,0.680492,NaN
562,1952,Australia,1.253677e+05,NaN,8.782430,3.591675,2130.956115,0.680492,NaN
563,1953,Australia,1.389683e+05,NaN,8.950892,3.653409,2111.249251,0.680492,NaN
564,1954,Australia,1.500782e+05,0.059969,9.159148,3.731083,2091.724634,0.680492,0.716984
...,...,...,...,...,...,...,...,...,...
12175,2015,United States,1.877616e+07,0.961123,320.878310,150.248474,1770.023174,0.595646,0.992893
12176,2016,United States,1.909750e+07,0.980157,323.015995,152.396957,1766.744125,0.593773,0.993724
12177,2017,United States,1.954298e+07,1.000000,325.084756,154.672318,1763.726676,0.596151,1.000000
12178,2018,United States,2.012858e+07,1.022971,327.096265,156.675903,1774.703811,0.594326,1.008166


In [ ]:
# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('country')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita

In [ ]:
# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N
"""
data = data.sort_values('year').groupby('country').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()
"""

"\ndata = data.sort_values('year').groupby('country').apply(lambda x: x.assign(\n    alpha=1 - x['labsh'],\n    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],\n    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],\n    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],\n    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]\n)).reset_index(drop=True).dropna()\n"

In [ ]:
# Remove missing values
# 1data = data.dropna()
data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()

# Calculate summary statistics
summary = data.groupby('country').agg('mean')

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']

# Print output
display(summary)

,year,rgdpna,rkna,pop,emp,avh,labsh,rtfpna,y_pc,k_pc,a,g,alpha,y_n,hours,tfp_term,cap_term,lab_term,Growth Rate,Capital Share
country,,,,,,,,,,,,,,,,,,,,
Australia,1980.000,4.354819e+05,0.255803,14.755316,6.396266,1900.288204,0.652714,0.850442,11.072803,-3.343945,0.347286,1.652004,0.347286,65546.593750,12102.855874,0.781351,0.000749,817.104158,1.652004,0.347286
Austria,1980.000,2.166596e+05,0.345234,7.625111,3.465632,1881.910990,0.630036,0.839461,10.979732,-2.437459,0.369964,3.098609,0.369964,61997.203125,6516.332598,0.764235,0.000388,855.926477,3.098609,0.369964
Belgium,1980.000,2.499445e+05,0.339617,9.812326,3.756693,1745.117573,0.636911,0.891169,11.051198,-2.502341,0.363089,2.685889,0.363089,66017.835938,6542.447849,0.844801,0.000460,668.450129,2.685889,0.363089
Canada,1980.000,7.430176e+05,0.289481,24.416506,10.654404,1869.552187,0.719970,0.958237,11.109036,-3.715119,0.280030,1.575284,0.280030,67515.687500,19695.690240,0.945138,0.005070,801.345674,1.575284,0.280030
Denmark,1980.000,1.543718e+05,0.365569,5.039878,2.489403,1638.720279,0.641831,0.854043,10.992245,-2.025134,0.358169,2.282366,0.358169,61140.757812,4050.745867,0.786575,0.000710,806.428892,2.282366,0.358169
Finland,1980.000,1.094319e+05,0.402294,4.819167,2.288248,1848.927383,0.664116,0.713644,10.706371,-1.849027,0.335884,3.461884,0.335884,47912.968750,4229.166526,0.614840,0.002486,881.944877,3.461884,0.335884
France,1980.000,1.436851e+06,0.385618,54.867686,22.355995,1828.887702,0.652998,0.825985,11.005585,-4.179371,0.347002,2.942555,0.347002,63180.828125,40614.251688,0.757989,0.000444,746.997081,2.942555,0.347002
Germany,1980.000,2.214914e+06,0.416528,78.407925,38.228958,1780.610503,0.665878,0.730588,10.916740,-4.633936,0.334122,2.811127,0.334122,57835.121094,68011.255862,0.631489,0.000418,870.112196,2.811127,0.334122
Greece,1980.000,1.814023e+05,0.434029,9.547269,3.763981,2126.534011,0.488920,1.155891,10.708709,-2.281558,0.511080,3.369869,0.511080,47431.316406,8000.180712,1.371518,0.000001,839.867634,3.369869,0.511080


In [ ]:
data

,year,country,rgdpna,rkna,pop,emp,avh,labsh,rtfpna,y_pc,k_pc,a,g,alpha,y_n,hours,tfp_term,cap_term,lab_term
8830,1960,Netherlands,1.839613e+05,0.125108,11.459214,4.709317,1855.119795,0.728939,0.612767,10.572938,-3.628120,0.271061,7.547569,0.271061,39063.277344,8736.346685,0.510740,5.088562e-03,762.386210
3440,1960,Denmark,7.579979e+04,0.126833,4.583308,2.111657,1909.468236,0.641629,0.666206,10.488378,-2.812357,0.358371,3.966045,0.358371,35895.878906,4032.142240,0.530992,5.938345e-04,879.744909
4770,1960,Greece,5.940977e+04,0.138729,8.205011,3.608476,2227.892576,0.488188,0.732547,9.708928,-3.258517,0.511812,1.853943,0.511812,16463.949219,8039.297253,0.528604,1.246755e-06,979.803389
3230,1960,Germany,1.092642e+06,0.130798,73.196345,37.446384,2181.886672,0.668121,0.523795,10.281199,-5.657013,0.331879,7.016945,0.331879,29178.845703,81703.767094,0.379891,3.644692e-04,1116.227414
2110,1960,Canada,3.104354e+05,0.091596,17.951266,6.629172,2090.805868,0.768659,0.795596,10.754251,-4.281843,0.231341,3.451920,0.231341,46828.691406,13860.311403,0.742685,1.083079e-02,772.107725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10830,2000,Sweden,3.555356e+05,0.685032,8.881640,4.305337,1643.438430,0.521596,0.912320,11.321526,-1.838144,0.478404,2.296638,0.478404,82580.203125,7075.556202,0.838678,5.729239e-06,796.649741
6140,2000,Japan,4.390924e+06,0.826871,127.524174,65.915504,1858.500000,0.581989,0.963764,11.106677,-4.378480,0.418011,3.509331,0.418011,66614.437500,122503.965031,0.938551,1.478122e-05,960.633276
5930,2000,Italy,2.372886e+06,0.773009,56.692178,22.917969,1850.197971,0.502191,1.185745,11.547696,-3.389386,0.497809,1.753998,0.497809,103538.242188,42402.779286,1.403902,3.711147e-07,747.947614
5790,2000,Iceland,1.001764e+04,0.635077,0.280435,0.157086,1695.384968,0.648463,0.799924,11.063067,1.396956,0.351537,3.224087,0.351537,63771.878906,266.320434,0.708746,5.299910e-03,949.669030
